In [ ]:
# Importing packages

import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import shutil
import random
import cv2

from PIL import Image
import pickle

import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))

physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)
tf.config.experimental.set_visible_devices(physical_devices[0], 'GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


**Building sequences of both images and irradiance values for train set and saving them in data_seq folder**

In [ ]:
#Using skycamera_images - Copie folder that contains sky images of each day, and labels_new folder that 
#contains corresponding irradiance values for each day to buil data_seq folder that will contains
#sequences of input data for the models (sequences of 2 hours long images and irradiance and their corresponding
#target which is sequence of the next 15 minutes of irradiance evolution)

def construct_seq_data(window_size=120,forecast_size=16,ecart=45,img_source_f,lbl_source_f):

        #Creating data_seq folder
        if os.path.exists("data_seq"):
                shutil.rmtree("data_seq")
        os.makedirs("data_seq")

        #Looping over each day so we can construct as many sequences as possible from each day
        for day_folder in os.listdir(img_source_f):
                L=os.listdir(os.path.join(img_source_f,day_folder))
                
                #Building sequences, but each one should be far from the other by ecart=45 minutes
                for i in range(0,len(L),ecart):
                        folder_name=day_folder+"_"+str(int(i/ecart))
                        img_subfold=(os.path.join("data_seq",folder_name,"images"))
                        
                        #Creating the int(i/ecart) subfolder that contain the int(i/ecart) sequence of that day
                        os.makedirs(img_subfold)

                        #Copy images to the data_seq folder for the specified day and sequence number of that day
                        for e in L[i:i+window_size+forecast_size]:
                                shutil.copy(os.path.join(img_source_f,day_folder,e),os.path.join(img_subfold))

                        #Check if the folder (especially the last ones) contains the expected number of images/labels because
                        # it has to be the same for all sequences because the model will be fed with a fix length size...
                        if len(os.listdir(img_subfold))!=136: #136 is 120 minutes + 16 for the target size
                                folder_name=day_folder+"_"+str(int(i/ecart))

                                #if the sequence size is not 136 (less than 136), we delete the corresponding folder
                                shutil.rmtree(img_subfold)

                                #We create a new one
                                file=(os.path.join("data_seq",folder_name,"images"))
                                os.makedirs(file)
                                
                                #For exemple if the sequence size is 100, we take the needed data (here 30) from
                                #the previous data of that same day to have 136 data point finally
                                for e in L[i-(window_size+forecast_size-(len(L)-i+1))-1:i+window_size+forecast_size]:
                                        shutil.copy(os.path.join(img_source_f,day_folder,e),os.path.join(img_subfold))
                
                                #Dping the same principle for txt data that contain irradiance values
                                file=(os.path.join("data_seq",folder_name,"labels"))
                                os.makedirs(file)
                                for e in L[i-(window_size+forecast_size-(len(L)-i+1))-1:i+window_size+forecast_size]:
                                        label_subfold=(os.path.join("data_seq",folder_name,"labels"))
                                        time=e.split(".")[0]
                                        label_file=time+".txt"
                                        shutil.copy(os.path.join(lbl_source_f,day_folder,label_file),os.path.join(label_subfold))

                                #Break the loop as further folders will be repetitions because we will always emprunt data from 
                                #previous sequences...
                                break 

                        #Creating labels subfolder for that same sequence folder of that same day
                        label_subfold=(os.path.join("data_seq",folder_name,"labels"))
                        os.makedirs(label_subfold)

                        #Copy the corresponding labels for the images
                        for e in L[i:i+window_size+forecast_size]:
                                time=e.split(".")[0]
                                label_file=time+".txt"
                                shutil.copy(os.path.join(lbl_source_f,day_folder,label_file),os.path.join(label_subfold))

                        #All the seq folders should have the same number of images/labels, so if it's not the case
                        #for the labels, we will apply the same principle applied before for the images
                        if len(os.listdir(label_subfold))!=136: 
                                folder_name=day_folder+"_"+str(int(i/ecart))
                                shutil.rmtree(label_subfold)
                                file=(os.path.join("data_seq",folder_name,"labels"))
                                os.makedirs(file)
                                
                                for e in L[i-(window_size+forecast_size-(len(L)-i+1))-1:i+window_size+forecast_size]:
                                        shutil.copy(os.path.join(lbl_source_f,day_folder,label_file),os.path.join(label_subfold)) 
                                
                                #Because those folders that will be created after will be just repetition
                                break 


img_source_f="skycamera_images - Copie"
lbl_source_f="labels_new"
window_size=120
forecast_size=16
ecart=45

construct_seq_data(window_size=120,forecast_size=16,ecart=45,img_source_f,lbl_source_f)
                
                

**Building sequences of both images and irradiance values for test sets and saving them in data_seq folder**

In [ ]:
#Creating test sequences with the same way as before. But this time using the source folders
#"Test_set_folder\\images" for images and "Test_set_folder\\labels" for labels, because those folders
#were the result of splitting the data into train/val/test that i did before

def construct_test_seq_data(window_size,forecast_size,ecart,img_source_f,lbl_source_f):

        #Creating test_seq folder
        if os.path.exists("test_seq"):
                shutil.rmtree("test_seq")
        os.makedirs("test_seq")

        for day_folder in os.listdir(img_source_f)[:10]:
                L=os.listdir(os.path.join(img_source_f,day_folder))
                for i in range(0,len(L),ecart):

                        folder_name=day_folder+"_"+str(int(i/ecart))
                        img_subfold=(os.path.join("test_seq",folder_name,"images"))
                        os.makedirs(img_subfold)

                        for e in L[i:i+window_size+forecast_size]:
                                shutil.copy(os.path.join(img_source_f,day_folder,e),os.path.join(img_subfold))

                        if len(os.listdir(img_subfold))!=136: # all the seq folders should have the same number of images/labels

                                folder_name=day_folder+"_"+str(int(i/ecart))
                                shutil.rmtree(img_subfold)
                                file=(os.path.join("test_seq",folder_name,"images"))
                                os.makedirs(file)
                                
                                for e in L[i-(window_size+forecast_size-(len(L)-i+1))-1:i+window_size+forecast_size]:
                                        shutil.copy(os.path.join(img_source_f,day_folder,e),os.path.join(img_subfold))
                
                                file=(os.path.join("test_seq",folder_name,"labels"))
                                os.makedirs(file)

                                for e in L[i-(window_size+forecast_size-(len(L)-i+1))-1:i+window_size+forecast_size]:
                                        label_subfold=(os.path.join("test_seq",folder_name,"labels"))
                                        time=e.split(".")[0]
                                        label_file=time+".txt"
                                        shutil.copy(os.path.join(lbl_source_f,day_folder,label_file),os.path.join(label_subfold))  
                
                                break #Because those folders that will be created after will be just repetition
                
                        label_subfold=(os.path.join("test_seq",folder_name,"labels"))
                        os.makedirs(label_subfold)
                
                        for e in L[i:i+window_size+forecast_size]:
                                time=e.split(".")[0]
                                label_file=time+".txt"

                                shutil.copy(os.path.join(lbl_source_f,day_folder,label_file),os.path.join(label_subfold))

                        if len(os.listdir(label_subfold))!=136: # all the seq folders should have the same number of images/labels

                                folder_name=day_folder+"_"+str(int(i/ecart))
                                shutil.rmtree(label_subfold)
                                file=(os.path.join("test_seq",folder_name,"labels"))
                                os.makedirs(file)
                                
                                for e in L[i-(window_size+forecast_size-(len(L)-i+1))-1:i+window_size+forecast_size]:
                                        shutil.copy(os.path.join(lbl_source_f,day_folder,label_file),os.path.join(label_subfold))

                                break #Because those folders that will be created after will be just repetition


img_source_f="Test_set_folder\\images"
lbl_source_f="Test_set_folder\\labels"
window_size=120
forecast_size=16
ecart=16

#The only difference here is that the ecart between two sequences is 16 like the target size.
#And that's because we will need after to concatenate all the predicted sequences of 16 minutes to build
#an entire day of predicted irradiance values
construct_test_seq_data(120,16,16,img_source_f,lbl_source_f)
                
                

**Dealing with an important issue**

In [ ]:
L=os.listdir("test_seq")
sorted(L)

#Here we will have a problem when testing a model. In fact, when feeding the model with this input data shown
#in terminal, the order will be not respected. The model will have in entry the _0 sequence, then _1 then _10
#instead of _2. There is no a problem but when rassembling all the resulted 15 minutes and trying to display
#them to compare the result and evaluate the model using metrics like MAE, MSE... (because to evaluate, we need
# actual irradiance and predicted one) so it will be hard to do so...
# So we need to make the sequences in order because for exemple if the _0 begins from minute 06:00 to 8:00
# then the _1 one will begin at 06:15 (ecart=16) to 08:15 and so on. So we have to respect the order.


['2021-07-02_0',
 '2021-07-02_1',
 '2021-07-02_10',
 '2021-07-02_11',
 '2021-07-02_12',
 '2021-07-02_13',
 '2021-07-02_14',
 '2021-07-02_15',
 '2021-07-02_16',
 '2021-07-02_17',
 '2021-07-02_18',
 '2021-07-02_19',
 '2021-07-02_2',
 '2021-07-02_20',
 '2021-07-02_21',
 '2021-07-02_22',
 '2021-07-02_23',
 '2021-07-02_24',
 '2021-07-02_25',
 '2021-07-02_26',
 '2021-07-02_27',
 '2021-07-02_28',
 '2021-07-02_29',
 '2021-07-02_3',
 '2021-07-02_30',
 '2021-07-02_31',
 '2021-07-02_32',
 '2021-07-02_33',
 '2021-07-02_34',
 '2021-07-02_35',
 '2021-07-02_36',
 '2021-07-02_37',
 '2021-07-02_38',
 '2021-07-02_39',
 '2021-07-02_4',
 '2021-07-02_40',
 '2021-07-02_41',
 '2021-07-02_5',
 '2021-07-02_6',
 '2021-07-02_7',
 '2021-07-02_8',
 '2021-07-02_9',
 '2021-07-08_0',
 '2021-07-08_1',
 '2021-07-08_10',
 '2021-07-08_11',
 '2021-07-08_12',
 '2021-07-08_13',
 '2021-07-08_14',
 '2021-07-08_15',
 '2021-07-08_16',
 '2021-07-08_17',
 '2021-07-08_18',
 '2021-07-08_19',
 '2021-07-08_2',
 '2021-07-08_20',
 '202

In [ ]:
L=[]
for el in os.listdir("test_seq"):
    pref=el.split("_")[0]
    num=el.split("_")[1]
    numm=bin(int(num))[2:]
    numm=str(numm)
    if len(numm)<6:
        a=len(numm)
        numm=(6-a)*"0"+numm
    print(pref+'_'+numm,num)
    L.append(pref+'_'+numm)

    new_path=pref+'_'+numm
    os.rename(os.path.join("test_seq",el),os.path.join("test_seq",new_path))

#The solution that i did is to rename the folders and replace the sequence number for all day folders
#by its binary form to respect the order...

2021-07-02_000000 0
2021-07-02_000001 1
2021-07-02_001010 10
2021-07-02_001011 11
2021-07-02_001100 12
2021-07-02_001101 13
2021-07-02_001110 14
2021-07-02_001111 15
2021-07-02_010000 16
2021-07-02_010001 17
2021-07-02_010010 18
2021-07-02_010011 19
2021-07-02_000010 2
2021-07-02_010100 20
2021-07-02_010101 21
2021-07-02_010110 22
2021-07-02_010111 23
2021-07-02_011000 24
2021-07-02_011001 25
2021-07-02_011010 26
2021-07-02_011011 27
2021-07-02_011100 28
2021-07-02_011101 29
2021-07-02_000011 3
2021-07-02_011110 30
2021-07-02_011111 31
2021-07-02_100000 32
2021-07-02_100001 33
2021-07-02_100010 34
2021-07-02_100011 35
2021-07-02_100100 36
2021-07-02_100101 37
2021-07-02_100110 38
2021-07-02_100111 39
2021-07-02_000100 4
2021-07-02_101000 40
2021-07-02_101001 41
2021-07-02_000101 5
2021-07-02_000110 6
2021-07-02_000111 7
2021-07-02_001000 8
2021-07-02_001001 9
2021-07-08_000000 0
2021-07-08_000001 1
2021-07-08_001010 10
2021-07-08_001011 11
2021-07-08_001100 12
2021-07-08_001101 13
2021

In [ ]:
sorted(L)

['2021-07-02_000000',
 '2021-07-02_000001',
 '2021-07-02_000010',
 '2021-07-02_000011',
 '2021-07-02_000100',
 '2021-07-02_000101',
 '2021-07-02_000110',
 '2021-07-02_000111',
 '2021-07-02_001000',
 '2021-07-02_001001',
 '2021-07-02_001010',
 '2021-07-02_001011',
 '2021-07-02_001100',
 '2021-07-02_001101',
 '2021-07-02_001110',
 '2021-07-02_001111',
 '2021-07-02_010000',
 '2021-07-02_010001',
 '2021-07-02_010010',
 '2021-07-02_010011',
 '2021-07-02_010100',
 '2021-07-02_010101',
 '2021-07-02_010110',
 '2021-07-02_010111',
 '2021-07-02_011000',
 '2021-07-02_011001',
 '2021-07-02_011010',
 '2021-07-02_011011',
 '2021-07-02_011100',
 '2021-07-02_011101',
 '2021-07-02_011110',
 '2021-07-02_011111',
 '2021-07-02_100000',
 '2021-07-02_100001',
 '2021-07-02_100010',
 '2021-07-02_100011',
 '2021-07-02_100100',
 '2021-07-02_100101',
 '2021-07-02_100110',
 '2021-07-02_100111',
 '2021-07-02_101000',
 '2021-07-02_101001',
 '2021-07-08_000000',
 '2021-07-08_000001',
 '2021-07-08_000010',
 '2021-07-

**Checking if the transformation is working with labels files also**

In [ ]:
import glob

def natural_sort(file_paths):
    """Sort file paths naturally."""
    import re
    return sorted(file_paths, key=lambda path: [int(s) if s.isdigit() else s for s in re.split(r'(\d+)', path)])

# Get the list of file paths in the desired order
file_pattern = "test_seq/*/labels/*"
file_paths = glob.glob(file_pattern)
sorted_file_paths = natural_sort(file_paths)

file_paths

['test_seq\\2021-07-02_0\\labels\\03-19.txt',
 'test_seq\\2021-07-02_0\\labels\\03-20.txt',
 'test_seq\\2021-07-02_0\\labels\\03-21.txt',
 'test_seq\\2021-07-02_0\\labels\\03-22.txt',
 'test_seq\\2021-07-02_0\\labels\\03-23.txt',
 'test_seq\\2021-07-02_0\\labels\\03-24.txt',
 'test_seq\\2021-07-02_0\\labels\\03-25.txt',
 'test_seq\\2021-07-02_0\\labels\\03-26.txt',
 'test_seq\\2021-07-02_0\\labels\\03-27.txt',
 'test_seq\\2021-07-02_0\\labels\\03-28.txt',
 'test_seq\\2021-07-02_0\\labels\\03-29.txt',
 'test_seq\\2021-07-02_0\\labels\\03-30.txt',
 'test_seq\\2021-07-02_0\\labels\\03-31.txt',
 'test_seq\\2021-07-02_0\\labels\\03-32.txt',
 'test_seq\\2021-07-02_0\\labels\\03-33.txt',
 'test_seq\\2021-07-02_0\\labels\\03-34.txt',
 'test_seq\\2021-07-02_0\\labels\\03-35.txt',
 'test_seq\\2021-07-02_0\\labels\\03-36.txt',
 'test_seq\\2021-07-02_0\\labels\\03-37.txt',
 'test_seq\\2021-07-02_0\\labels\\03-38.txt',
 'test_seq\\2021-07-02_0\\labels\\03-39.txt',
 'test_seq\\2021-07-02_0\\labels\\

**Building sequences of both images and irradiance values for validation set and saving them in data_seq folder**

In [ ]:
def construct_seq_val_data(img_source_f,lbl_source_f,window_size,forecast_size,ecart):

        if os.path.exists("val_data_seq"):
                shutil.rmtree("val_data_seq")
        os.makedirs("val_data_seq")
        for day_folder in os.listdir(img_source_f):
                L=os.listdir(os.path.join(img_source_f,day_folder))
                for i in range(0,len(L),ecart):
                  
                        folder_name=day_folder+"_"+str(int(i/ecart))
                        img_subfold=(os.path.join("val_data_seq",folder_name,"images"))
                        os.makedirs(img_subfold)
                        for e in L[i:i+window_size+forecast_size]:
                                shutil.copy(os.path.join(img_source_f,day_folder,e),os.path.join(img_subfold))
                        if len(os.listdir(img_subfold))!=136: # all the seq folders should have the same number of images/labels
                                folder_name=day_folder+"_"+str(int(i/ecart))
                                shutil.rmtree(img_subfold)
                                file=(os.path.join("val_data_seq",folder_name,"images"))
                                os.makedirs(file)
                                
                                for e in L[i-(window_size+forecast_size-(len(L)-i+1))-1:i+window_size+forecast_size]:
                                        shutil.copy(os.path.join(img_source_f,day_folder,e),os.path.join(img_subfold))
                
                                #shutil.rmtree(label_subfold)
                                file=(os.path.join("val_data_seq",folder_name,"labels"))
                                os.makedirs(file)
        
                                for e in L[i-(window_size+forecast_size-(len(L)-i+1))-1:i+window_size+forecast_size]:
                                        label_subfold=(os.path.join("val_data_seq",folder_name,"labels"))
                                        time=e.split(".")[0]
                                        label_file=time+".txt"
                                        shutil.copy(os.path.join(lbl_source_f,day_folder,label_file),os.path.join(label_subfold))  
                
                                break #Because those folders that will be created after will be just repetition
                
                        label_subfold=(os.path.join("val_data_seq",folder_name,"labels"))
                        os.makedirs(label_subfold)
                
                        for e in L[i:i+window_size+forecast_size]:
                                time=e.split(".")[0]
                                label_file=time+".txt"
                                shutil.copy(os.path.join(lbl_source_f,day_folder,label_file),os.path.join(label_subfold))
                        if len(os.listdir(label_subfold))!=136: # all the seq folders should have the same number of images/labels
                                folder_name=day_folder+"_"+str(int(i/ecart))
                                shutil.rmtree(label_subfold)
                                file=(os.path.join("val_data_seq",folder_name,"labels"))
                                os.makedirs(file)
                                #print(i,i-(window_size+forecast_size-(len(L)-i+1))-1,i+window_size+forecast_size)
                                for e in L[i-(window_size+forecast_size-(len(L)-i+1))-1:i+window_size+forecast_size]:
                                        shutil.copy(os.path.join(lbl_source_f,day_folder,label_file),os.path.join(label_subfold))  
                                break #Because those folders that will be created after will be just repetition


img_source_f="Val_set_folder\\images"
lbl_source_f="Val_set_folder\\labels"
window_size=120
forecast_size=16
ecart=45
construct_seq_val_data(img_source_f,lbl_source_f,window_size=120,forecast_size=16,ecart=45)